En este documento exploraremos y analizaremos un dataset proporcionado por el profesorado que imparte la asignatura _Aprendizaje Automático_ en la Facultad de Informática, Universidad Complutense de Madrid. Implementaremos un modelo que realice predicciones de etiquetado binario utilizando este dataset, considerando múltiples configuraciones para el modelo, observando cómo varía su precisión y eficiencia. Para llevar a cabo estas tareas, utilizaremos las librerías _matplotlib, numpy y scikit-learn_.

El objetivo de este ejercicio es, por lo tanto, poner en práctica varias habilidades aprendidas durante el desarrollo de la asignatura a lo largo de los últimos meses. De esta forma, utilizaremos únicamente metodologías impartidas en la asignatura.

El dataset que utilizaremos para desarrollar nuestro proyecto pertenece a [_UC Irvine Machine Learning Repository_](http://archive.ics.uci.edu/ml/index.php) y se denomina [Tic-Tac-Toe Endgame](http://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame).


A continuación, realizaremos la carga de librerías, datos y un breve análisis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt